In [ ]:
from pathlib import Path
import sqlite3
from collections import namedtuple

import numpy as np
import sklearn
import pandas as pd
from pandas import Int64Index

In [ ]:
path_data = Path('sql')
path_data.mkdir(exist_ok=True)
db_file = path_data / 'db.sqlite'
with sqlite3.connect(str(db_file)) as con:
    df = pd.read_sql(con=con, sql='select s.article_id_1, s.article_id_2, s.permid, s.sklearn, a1.source_id as source_id_1, a2.source_id as source_id_2 from similarities s inner join article a1 on s.article_id_1 = a1.article_id inner join article a2 on s.article_id_2 = a2.article_id')
df.head(5)

In [ ]:
threshold = 0.5
df = df[df.sklearn > threshold]
df = df.sort_values('sklearn', ascending=False)
df = df[df.source_id_1 != df.source_id_2]
df = df.reset_index(drop=True)
# df.article_id_1 = df.article_id_1.astype(int)
# df.article_id_2 = df.article_id_2.astype(int)
# df.source_id_1 = df.source_id_1.astype(int)
# df.source_id_2 = df.source_id_2.astype(int)
df.head(5)

In [ ]:
cliques = []
while len(cliques) < 2 and len(df) >= 4:
    c = [None, None, None, None]

    first_row = df.iloc[0]
    df = df.drop(df.head(1).index)

    c[int(first_row.source_id_1) - 1] = int(first_row.article_id_1)
    c[int(first_row.source_id_2) - 1] = int(first_row.article_id_2)
    c
    index = 0

    while c[0] is None or c[1] is None or c[2] is None or c[3] is None and len(df) > index:
        df = df.reset_index(drop=True)
#         print(index)
        new_row = df.iloc[index]
        if c[int(new_row.source_id_1) - 1] is None and c[int(new_row.source_id_2) - 1] is not None:
            c[int(new_row.source_id_1) - 1] = int(new_row.article_id_1)
            df = df.drop(Int64Index([index+1], dtype='int64'))
        elif c[int(new_row.source_id_1) - 1] is not None and c[int(new_row.source_id_2) - 1] is None:
            c[int(new_row.source_id_2) - 1] = int(new_row.article_id_2)
            df = df.drop(Int64Index([index+1], dtype='int64'))
        else:
            index += 1
    for i in range(4):
        df = df[(df.article_id_1 != c[i]) & (df.article_id_2 != c[i])]
    cliques.append(c)
print(cliques)
    

In [ ]:
with sqlite3.connect(str(db_file)) as con:
    cur = con.cursor()
    cur.execute('DELETE FROM matching')
    [cur.execute('INSERT INTO matching (article_id, group_id) VALUES (?, ?)', (a, i))
     for i, c in enumerate(cliques)
     for a in c]
    con.commit()